# Introduction

***As a data analyst, the task is to work with an ecommerce company in India (referred to as X) that receives a thousand orders
daily through their website and must deliver them promptly. To fulfill this task, X has partnered with multiple courier
companies in India, which charge them per delivery. The project's aim is to analyze the data to identify patterns andinsights to optimize delivery processes and costs.***

For this project, the client (X) is an ecommerce company in India that ships a large volume of orders on a daily basis. The charges for shipping each order are dependent upon two factors:

**1. The weight of the product and**

**2. The distance between the warehouse (pickup location) and the customer's delivery address (destination location).**

On average, the delivery charges are Rs. 100 per shipment. X ships 1,00,000 orders per month, which means they have to pay approximately Rs. 1 crore to the courier companies on a monthly basis as charges. Given the high cost of these charges, X wants to verify the accuracy of the charges levied by the courier companies to ensure that they are being charged correctly.

# ***Import Libraries***

In [1]:
import pandas as pd
import numpy as np
import math

## ***Data Description***

The objective of this challenge is to perform an analysis on five Excel files containing data from both Company ABC and Courier Company. The aim of the analysis is to verify the accuracy of the charges levied by the courier partners per order. The analysis will involve examining the data to ensure that the charges are being correctly calculated and applied to each order. The analysis will be conducted with the goal of providing assurance to Company ABC that they are not being overcharged and that the charges are consistent with the terms of their agreement with the courier partners.

## ***Load 'X' Company DataSet***

In [2]:
E_Com_order_report = pd.read_excel('Company X - Order Report.xlsx') # Website order report detail file
E_Com_order_report =E_Com_order_report.rename(columns = {'ExternOrderNo':'Order ID'}) #Rename ExternOrderNo into Order ID to differentiate
E_Com_order_report.head()

,Order ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [3]:
E_Com_pincode_zone = pd.read_excel('Company X - Pincode Zones.xlsx') #All India pincode Detail file
E_Com_pincode_zone = E_Com_pincode_zone.rename(columns ={'Zone': 'Delivery zone as per X company'}) # Rename Columns name differentiate
E_Com_pincode_zone.duplicated().sum() # Check Duplicates values
E_Com_pincode_zone = E_Com_pincode_zone.drop_duplicates(subset = 'Customer Pincode') # Remove Duplicates values
E_Com_pincode_zone.head()

,Warehouse Pincode,Customer Pincode,Delivery zone as per X company
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [4]:
E_Com_SKU_master = pd.read_excel('Company X - SKU Master.xlsx') #SKU master with gross weight of each product File
E_Com_SKU_master.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


# ***Courier Company Data Load***

In [5]:
Courier_Company_invoice = pd.read_excel('Courier Company - Invoice.xlsx') # Courier Company Invoice in Excel file)
# Rename Columns name to differentiate 
Courier_Company_invoice = Courier_Company_invoice.rename(columns ={'Zone': 'Delivery zone as per courier company','Charged Weight':'Total Weight As Per Courier Company (KG)'})
Courier_Company_invoice.head()

,AWB Code,Order ID,Total Weight As Per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery zone as per courier company,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [6]:
Courier_Company_rates = pd.read_excel('Courier Company - Rates.xlsx') #Courier Charges Rate Card
Courier_Company_rates.head()

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [7]:
print(Courier_Company_rates.columns)

Index(['fwd_a_fixed', 'fwd_a_additional', 'fwd_b_fixed', 'fwd_b_additional',
       'fwd_c_fixed', 'fwd_c_additional', 'fwd_d_fixed', 'fwd_d_additional',
       'fwd_e_fixed', 'fwd_e_additional', 'rto_a_fixed', 'rto_a_additional',
       'rto_b_fixed', 'rto_b_additional', 'rto_c_fixed', 'rto_c_additional',
       'rto_d_fixed', 'rto_d_additional', 'rto_e_fixed', 'rto_e_additional'],
      dtype='object')


## ***Exploratory Data Analysis***

In [8]:
mrg_order_SKU = E_Com_order_report.merge(E_Com_SKU_master, how = 'left', on = 'SKU') # Merge Website order report and SKU master with gross weight
mrg_order_SKU.head()

,Order ID,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150
2,2001827036,8904223819109,1.0,100
3,2001827036,8904223818430,1.0,165
4,2001827036,8904223819277,1.0,350


In [9]:
E_Com_pincode_zone = E_Com_pincode_zone.drop_duplicates(subset=['Customer Pincode']) # Drop Duplicates values from pincode file
Courier_Company_invoice_X = Courier_Company_invoice[['Order ID', 'Customer Pincode','Type of Shipment']] #Extract Cloumns From Courier Company Invoice
mrg_Pincode_Courier = Courier_Company_invoice_X.merge(E_Com_pincode_zone, on = 'Customer Pincode') # Merge Courier Company Invoice and pincode file
mrg_Pincode_Courier.sample()

,Order ID,Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per X company
11,2001807814,284001,Forward charges,121003,b


In [10]:
mrg_X_Courier = mrg_order_SKU.merge(mrg_Pincode_Courier, on = 'Order ID') # Merge mrg_order_SKU and mrg_Pincode_Courier file
mrg_X_Courier.sample()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per X company
108,2001811363,8904223815859,1.0,165,321608,Forward charges,121003,b


In [11]:
mrg_X_Courier=mrg_X_Courier.rename(columns={"Order Qty":"Order_Qty","Weight (g)":"Weight_g"}) # Rename Columns Name
mrg_X_Courier.sample()

,Order ID,SKU,Order_Qty,Weight_g,Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per X company
228,2001808286,8904223818683,1.0,121,302031,Forward charges,121003,b


In [12]:
# Creating a new feature for find Total weight as per shipment
mrg_X_Courier['Total_Weight_Per_Shipment'] = Total_Weight_Per_Shipment=(mrg_X_Courier['Order_Qty']*mrg_X_Courier['Weight_g'])/1000
mrg_X_Courier.sample()

,Order ID,SKU,Order_Qty,Weight_g,Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per X company,Total_Weight_Per_Shipment
32,2001821190,8904223819321,1.0,600,486661,Forward and RTO charges,121003,d,0.6


In [13]:
#renaming columns
mrg_X_Courier=mrg_X_Courier.rename(columns={"Order_Qty":"Order Qty","Total_Weight_Per_Shipment":"Total Weight AS Per X (KG)","Weight_g":"Weight (g)"})
mrg_X_Courier.sample()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per X company,Total Weight AS Per X (KG)
231,2001808207,8904223818706,1.0,127,495671,Forward charges,121003,d,0.127


## ***Function For Creating Weight Slab***

In [14]:
# Creating a function to finding a weight slab
import math 
def weight_slab(weight):
    return math.ceil(weight*2)/2

## ***Generation Of Weight Slab Column***

In [15]:
# Creating a new fearture for Weight Slab
mrg_X_Courier['Weight Slab As Per x (KG)']=mrg_X_Courier['Total Weight AS Per X (KG)'].apply(weight_slab)
mrg_X_Courier.sample()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per X company,Total Weight AS Per X (KG),Weight Slab As Per x (KG)
27,2001821502,8904223818980,1.0,110,314401,Forward charges,121003,b,0.11,0.5


In [16]:
# Creating a new fearture for Weight Slab Charged By Courier Company (KG)
Courier_Company_invoice['Weight Slab Charged By Courier Company (KG)']=(Courier_Company_invoice['Total Weight As Per Courier Company (KG)']).apply(weight_slab)
Courier_Company_invoice.sample()

,AWB Code,Order ID,Total Weight As Per Courier Company (KG),Warehouse Pincode,Customer Pincode,Delivery zone as per courier company,Type of Shipment,Billing Amount (Rs.),Weight Slab Charged By Courier Company (KG)
63,1091117437680,2001808801,0.8,121003,335001,d,Forward charges,90.2,1.0


## ***Computation Of Expected Charge***

In [17]:
# Initialize the Expected_Charges list with NaN values
Expected_Charges = []

# Loop through the rows of the DataFrame and fill in the Expected_Charges values
for i in range(len(mrg_X_Courier)):
    fwd_catagory = 'fwd_'+mrg_X_Courier['Delivery zone as per X company'][i]
    fwd_fixed = Courier_Company_rates[fwd_catagory+'_fixed'][0]
    fwd_additional = Courier_Company_rates[fwd_catagory+'_additional'][0]
    rto_catagory = 'rto_'+mrg_X_Courier['Delivery zone as per X company'][i]
    rto_fixd = Courier_Company_rates[rto_catagory+'_fixed'][0]
    rto_additional = Courier_Company_rates[rto_catagory+'_additional'][0]
    if mrg_X_Courier['Type of Shipment'][i] == 'Forward charges':
        if mrg_X_Courier['Weight Slab As Per x (KG)'][i] <= 0.5:
            Expected_Charges.append(fwd_fixed)
        else:
            additional_weight = (mrg_X_Courier['Weight Slab As Per x (KG)'][i]-0.5)/0.5
            Expected_Charges.append(fwd_fixed+additional_weight*fwd_additional)
    elif mrg_X_Courier['Type of Shipment'][i] == 'Forward and RTO charges':
        if mrg_X_Courier['Weight Slab As Per x (KG)'][i] <= 0.5:
            Expected_Charges.append(fwd_fixed+rto_fixd)
        else:
            additional_weight=(mrg_X_Courier['Weight Slab As Per x (KG)'][i]-0.5)/0.5
            Expected_Charges.append(fwd_fixed+additional_weight*(fwd_additional+rto_additional))

# Assign the Expected_Charges list to the 'Expected Charge As Per X (Rs.)' column of the DataFrame
mrg_X_Courier['Expected Charge As Per X (Rs.)'] = Expected_Charges
mrg_X_Courier.sample()

,Order ID,SKU,Order Qty,Weight (g),Customer Pincode,Type of Shipment,Warehouse Pincode,Delivery zone as per X company,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.)
211,2001808675,8904223815859,1.0,165,226010,Forward charges,121003,b,0.165,0.5,33.0


In [18]:
Output2=mrg_X_Courier[['Order ID',
       'Type of Shipment', 'Delivery zone as per X company',
       'Total Weight AS Per X (KG)', 'Weight Slab As Per x (KG)',
       'Expected Charge As Per X (Rs.)']]

In [19]:
Courier_Company_invoice.columns

Index(['AWB Code', 'Order ID', 'Total Weight As Per Courier Company (KG)',
       'Warehouse Pincode', 'Customer Pincode',
       'Delivery zone as per courier company', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Weight Slab Charged By Courier Company (KG)'],
      dtype='object')

In [20]:
Output1=Courier_Company_invoice[['AWB Code', 'Order ID','Total Weight As Per Courier Company (KG)',
        'Delivery zone as per courier company','Billing Amount (Rs.)', 'Weight Slab Charged By Courier Company (KG)']]

In [21]:
df_merged_output=Output2.merge(Output1,on='Order ID',how='left')
df_merged_output.sample(5)

,Order ID,Type of Shipment,Delivery zone as per X company,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Code,Total Weight As Per Courier Company (KG),Delivery zone as per courier company,Billing Amount (Rs.),Weight Slab Charged By Courier Company (KG)
188,2001809270,Forward charges,b,0.133,0.5,33.0,1091117613962,0.68,d,90.2,1.0
108,2001811363,Forward charges,b,0.165,0.5,33.0,1091117958395,0.59,d,90.2,1.0
309,2001807036,Forward charges,d,0.115,0.5,45.4,1091117323005,1.64,d,179.8,2.0
160,2001809820,Forward charges,d,0.350,0.5,45.4,1091117795623,3.00,d,269.4,3.0
219,2001808542,Forward charges,d,0.254,0.5,45.4,1091117436464,0.86,d,90.2,1.0


In [22]:
resultant_output1=df_merged_output.copy()

In [23]:
resultant_output1=resultant_output1.rename(columns={'AWB Code':'AWB Number','Billing Amount (Rs.)':'Charges_Billed_By_Courier_Company_Rs','Expected Charge As Per ABC (Rs.)':'Expected_Charge_As_Per_ABC_Rs'})
resultant_output1.head()

,Order ID,Type of Shipment,Delivery zone as per X company,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Number,Total Weight As Per Courier Company (KG),Delivery zone as per courier company,Charges_Billed_By_Courier_Company_Rs,Weight Slab Charged By Courier Company (KG)
0,2001827036,Forward charges,e,0.127,0.5,56.6,1091122418320,1.6,b,117.9,2.0
1,2001827036,Forward charges,e,0.150,0.5,56.6,1091122418320,1.6,b,117.9,2.0
2,2001827036,Forward charges,e,0.100,0.5,56.6,1091122418320,1.6,b,117.9,2.0
3,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0
4,2001827036,Forward charges,e,0.350,0.5,56.6,1091122418320,1.6,b,117.9,2.0


## ***Computation Of Difference Between Expected Charges And Billed Charges***

In [24]:
# Calculate the difference between the expected charges and billed charges
Difference_Between_Expected_Charges_and_Billed_Charges_Rs = resultant_output1['Expected Charge As Per X (Rs.)'] - resultant_output1['Charges_Billed_By_Courier_Company_Rs']

# Add the calculated difference as a new column in the DataFrame
resultant_output1['Difference Between Expected Charges and Billed Charges (Rs.)'] = Difference_Between_Expected_Charges_and_Billed_Charges_Rs

In [25]:
resultant_output1.head()

,Order ID,Type of Shipment,Delivery zone as per X company,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Number,Total Weight As Per Courier Company (KG),Delivery zone as per courier company,Charges_Billed_By_Courier_Company_Rs,Weight Slab Charged By Courier Company (KG),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,Forward charges,e,0.127,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
1,2001827036,Forward charges,e,0.150,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
2,2001827036,Forward charges,e,0.100,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
3,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
4,2001827036,Forward charges,e,0.350,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3


In [26]:
# Rename Columns name
resultant_output1=resultant_output1.rename(columns={'Charges_Billed_By_Courier_Company_Rs':'Charges Billed By Courier Company (Rs.)'})

In [27]:
resultant_output1.head()

,Order ID,Type of Shipment,Delivery zone as per X company,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Number,Total Weight As Per Courier Company (KG),Delivery zone as per courier company,Charges Billed By Courier Company (Rs.),Weight Slab Charged By Courier Company (KG),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,Forward charges,e,0.127,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
1,2001827036,Forward charges,e,0.150,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
2,2001827036,Forward charges,e,0.100,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
3,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3
4,2001827036,Forward charges,e,0.350,0.5,56.6,1091122418320,1.6,b,117.9,2.0,-61.3


In [28]:
writer = pd.ExcelWriter('Expected_Result.xlsx', engine='openpyxl')

In [29]:
# Write the DataFrame to the Excel file
resultant_output1.to_excel(writer, sheet_name='Calculations', index=False)
Calculations=writer.save()

## ***Computation Of Total Orders***

In [30]:
# Here we get Correct charges
correctly_charged = resultant_output1[resultant_output1['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0.0]
correctly = ['Total Orders correctly charged', len(correctly_charged), correctly_charged['Charges Billed By Courier Company (Rs.)'].sum()]


In [31]:
# Here we get Over charges 
over_charged = resultant_output1[resultant_output1['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0.0]
over = ['Total Orders overcharged', len(over_charged), abs(over_charged['Difference Between Expected Charges and Billed Charges (Rs.)'].sum())]


In [32]:
# Here we get Under charges 
under_charged = resultant_output1[resultant_output1['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0.0]
under = ['Total Orders undercharged', len(under_charged), abs(under_charged['Difference Between Expected Charges and Billed Charges (Rs.)'].sum())]

In [33]:
# Creating DataFreame 
resultant_output2=pd.DataFrame([correctly,over,under],columns=['Description','Count','Amount (Rs.)'])

In [34]:
resultant_output2

,Description,Count,Amount (Rs.)
0,Total Orders correctly charged,12,507.6
1,Total Orders overcharged,380,31737.5
2,Total Orders undercharged,9,165.2


## Generation Of Resultant Summary Output File

In [35]:
resultant_output2.to_excel(writer,sheet_name='Summary')
writer.save()

In [36]:
df_sheet_multi = pd.read_excel('Expected_Result.xlsx', sheet_name=['Calculations', 'Summary'])
df_sheet_multi['Calculations']

,Order ID,Type of Shipment,Delivery zone as per X company,Total Weight AS Per X (KG),Weight Slab As Per x (KG),Expected Charge As Per X (Rs.),AWB Number,Total Weight As Per Courier Company (KG),Delivery zone as per courier company,Charges Billed By Courier Company (Rs.),Weight Slab Charged By Courier Company (KG),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001827036,Forward charges,e,0.127,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
1,2001827036,Forward charges,e,0.150,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
2,2001827036,Forward charges,e,0.100,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
3,2001827036,Forward charges,e,0.165,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
4,2001827036,Forward charges,e,0.350,0.5,56.6,1091122418320,1.60,b,117.9,2.0,-61.3
...,...,...,...,...,...,...,...,...,...,...,...,...
396,2001806229,Forward charges,d,0.133,0.5,45.4,1091117222080,0.71,d,90.2,1.0,-44.8
397,2001806229,Forward charges,d,0.240,0.5,45.4,1091117222080,0.71,d,90.2,1.0,-44.8
398,2001806226,Forward charges,d,0.480,0.5,45.4,1091117222065,0.68,d,90.2,1.0,-44.8
399,2001806210,Forward charges,b,0.120,0.5,33.0,1091117221940,2.92,b,174.5,3.0,-141.5


In [37]:
df_sheet_multi['Summary']

,Unnamed: 0,Description,Count,Amount (Rs.)
0,0,Total Orders correctly charged,12,507.6
1,1,Total Orders overcharged,380,31737.5
2,2,Total Orders undercharged,9,165.2
